<a href="https://colab.research.google.com/github/elizamelo/SemanaFisica2025/blob/main/hepdata_semanafisica25_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---
# Hands-On: Análise de Dados


---


Parte I - ROOT

7. Fazer um ajuste no pico que corresponde ao méson $J/\Psi$ usando o RooFit.

8. Extrair as informações dos parâmetros.

9. Validação do fit (pull e/ou $\chi^{2}/ndof$).










In [ ]:
%load_ext autoreload
%autoreload 2

# Instalar: coffea, uproot, awkward

coffea, uproot, e awkward são três bibliotecas importantes para análise de dados em Física de Altas Energias (HEP), especialmente voltadas para trabalhar com arquivos ROOT, que são utilizados no contexto de experimentos no CERN.


* [coffea](https://github.com/CoffeaTeam/coffea
)

* [uproot](https://uproot.readthedocs.io/en/latest/basic.html)

*   [awkward](https://awkward-array.org/doc/main/)

* Ferramentas adicionais:
    * **pandas**: biblioteca para estruturas de dados tabulares e ferramentas de análise em Python.( [documentação do pandas](https://pandas.pydata.org/docs/getting_started/index.html))

  * **numpy**: fornece cálculos numéricos, como criação de histogramas.

  * **matplotlib**: ferramenta comum para criar gráficos, figuras, imagens, visualizações.

  * **vector**: é especializada no trabalho com vetores em 2D, 3D e vetores de Lorentz. Esta biblioteca foi projetada para funcionar com versões do Python 3.8 ou superiores.([documentação do vector](https://vector.readthedocs.io/en/latest/))



In [ ]:
try:
  import coffea
except ImportError as e:
  !pip install coffea
  import coffea
print (coffea.__version__)

In [ ]:
#!pip install uproot awkward xrootd
#!pip install fsspec-xrootd
import uproot
import awkward as ak
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import vector
import hist
import mplhep as hep

print (uproot.__version__)
print (ak.__version__)

# Baixar o dataframe

# Abrir e explorar o arquivo ROOT usando o uproot



* (1.) Abrir o arquivo.
* (2.) Verificar o conteúdo do arquivo.
* (3.) Verificar o conteúdo da tree: o número de branches, os nomes e os tipos de variáveis em cada branch.

In [ ]:
#Usando o output da Parte I ("muon_data_tree.root"
!wget -nc --no-check-certificate https://eliza.web.cern.ch/eliza/HandsOn/muon_data_tree.root


In [ ]:
# Abrir o arquivo ROOT para leitura
file = uproot.open("muon_data_tree.root")


print(file.keys())
tree = file["tree"]
print(tree.keys())  # Lista as branches (colunas)

# Ler algumas entradas para verificar se os dados estão corretos
df_r = tree.arrays(["muon1_pt", "muon1_eta", "muon1_phi", "pt_dimu"], library="pd")  # Carregar em um DataFrame
print(df_r.head())  # Visualizar as primeiras linhas do DataFrame


In [ ]:
#df_r = tree.arrays(tree.keys(), entry_stop=10000, library='pd')
#df_r

In [ ]:
#len(df_r)

Plots algumas varáveis cinemáticas do par

In [ ]:
#pt_dimu = df_r["pt_dimu"]


In [ ]:
'''
plt.figure()
plt.hist(pt_dimu, bins=100, range=(0, 200), histtype='step', color='blue', label=r'$p_T$ (GeV)')
plt.xlabel(r'$p_{T}^{\mu^+\mu^-} [GeV]/c$')
plt.ylabel('Events')
plt.yscale('log')
plt.show()
'''

In [ ]:
#mass_doismuons = df_r["mass_dimu"]


In [ ]:
#
'''
plt.figure()
plt.hist(mass_doismuons, bins=3000, range=(0.25, 300), histtype='step', color='blue', linewidth=1.5)
plt.xlabel(r'M$_{\mu^+\mu^-}$ [GeV]/$c^{2}$')
plt.ylabel('Events')
plt.yscale('log')
plt.xscale('log')
plt.show()
'''

#*Usando o ROOT e a sua ferramenta RooFit*




*   Baixar o Root e obter as dependências;
*   Importar as funções;
*   Com o plot da massa invariante:
  *   Escolher um pico de sinal de física;
  *   Os modelos para o sinal e fundo;
  *   Ajustar aos dados;
  *   Extrair informações;
  *   Validar o ajuste;








In [ ]:
!pip install -q condacolab


In [ ]:
import condacolab


In [ ]:
condacolab.install()


In [ ]:
!conda --version


In [ ]:
!conda install ROOT
#import ROOT as rt


In [ ]:
import ROOT

In [ ]:
# Importar as funções do ROOT aqui
from ROOT import TH1F
from ROOT import TFile
from ROOT import TLorentzVector
from ROOT import TLegend
from ROOT import RooFit
from ROOT import RooDataHist
from ROOT import RooDataSet
from ROOT import RooRealVar
from ROOT import RooArgList
from ROOT import RooArgSet
from ROOT import RooAddPdf
from ROOT import RooExponential
from ROOT import RooGaussian
from ROOT import RooPlot
from ROOT import RooCBShape
from ROOT import RooChi2Var
from ROOT import TLatex


O comando abaixo ativa o processamento multi-thread em ROOT, o que pode acelerar operações que envolvem grandes volumes de dados.

In [ ]:
ROOT.ROOT.EnableImplicitMT(4)  # Para usar 4 threads ou

In [ ]:
df_r = tree.arrays(["muon1_pt", "muon1_eta", "muon1_phi", "pt_dimu", "mass_dimu"], library="pd")  # Carregar em um DataFrame
print(df_r.head())  # Visualizar as primeiras linhas do DataFrame


In [ ]:
# Obter a variável mass_dimu a partir do DataFrame
mass_dimu = df_r["mass_dimu"]

In [ ]:
# Nome único para cada execução para evitar conflitos
hist_name = "invariant_mass_histogram"
canvas_name = "invariant_mass_canvas"

# Verifica e remove o histograma existente
if ROOT.gROOT.FindObject(hist_name):
    ROOT.gROOT.FindObject(hist_name).Delete()

# Verifica e remove o canvas existente
if ROOT.gROOT.FindObject(canvas_name):
    ROOT.gROOT.FindObject(canvas_name).Delete()

# Criação do histograma com ROOT
bins = 30000
low = 0.25
up = 300.0
hist = ROOT.TH1F(hist_name, r';Mass_{#mu#mu}(GeV/c^2);N_{Events}',bins , low, up)

hist.GetXaxis().SetTitleSize(0.042)
hist.GetYaxis().SetTitleSize(0.042)
hist.SetStats(False)

# Preenchendo o histograma com os dados de massa invariante
for mass in mass_dimu:
    hist.Fill(mass)

# Criação do canvas e desenho do histograma
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetTextFont(42)
canvas = ROOT.TCanvas(canvas_name, "Invariant Mass", 800, 700)
canvas.SetLogx()
canvas.SetLogy()
hist.Draw()

# Atualiza o canvas para garantir que o gráfico seja exibido
canvas.Update()
'''
label = ROOT.TLatex()
label.SetTextAlign(22)
#label.DrawLatex(0.55, 3.0e3, "#eta")
label.DrawLatex(0.77, 3.0e2, "#rho,#omega")
label.DrawLatex(1.20, 4.0e2, "#phi")
label.DrawLatex(4.00, 7.0e2, "J/#psi")
label.DrawLatex(4.60, 1.0e2, "#psi'")
label.DrawLatex(13.0, 0.9e2, "Y(1,2,3S)")
label.DrawLatex(91.0, 6.5e2, "Z")
label.SetNDC(True)
label.SetTextAlign(11)
label.SetTextSize(0.03)
label.DrawLatex(0.10, 0.92, "#bf{CMS Open Data}")
label.SetTextAlign(31)
label.DrawLatex(0.90, 0.92, "#sqrt{s} = 13 TeV, L_{int} = 4.7 fb^{-1}");
'''

canvas.Draw()

# Caso queira salvar o resultado
canvas.SaveAs("invariant_mass.png")

In [ ]:
#f = TFile("muon_data_tree.root")
#f.ls()


In [ ]:
#f.tree.Print()

In [ ]:
#max_Events = f.tree.GetEntries()
#max_Events

In [ ]:
'''
hist2 = ROOT.TH1F("", ";Mass_{#mu#mu}(GeV/c^2);N_{Events}",bins , low, up)

hist2.GetXaxis().SetTitleSize(0.040)
hist2.GetYaxis().SetTitleSize(0.040)
hist2.SetStats(False)

# Preenchendo o histograma com os dados de massa invariante
for i, mass in enumerate(f.tree):
    hist2.Fill(mass.mass_dimu)
    if i>max_Events:
      break

# Criação do canvas e desenho do histograma
canvas2 = ROOT.TCanvas("canvas2", "Invariant Mass", 800, 700)

hist2.Draw()
# Atualiza o canvas para garantir que o gráfico seja exibido
canvas2.Update()
'''

In [ ]:
#Configurar os limites da massa do pico escolhido, como exemplos usaremos o pico do J/Psi->muon+muon
#Com o hist sem o log em y
jpsi_pdg_mass =3.0969 #3096.900/1000
m_min = 2.9
m_max = 3.3
jpsi_bins = int((m_max - m_min)/0.05)
print(jpsi_bins)

# Definindo a variável de massa para RooFit
#Create a Mass variable that RooFit can use, and importing the relevant dataset
mass = ROOT.RooRealVar("mass", "m_{\mu^{+}\mu^{-}}", m_min, m_max, "GeV/c^{2}")
# Convertendo o histograma em um dataset do RooFit
data = ROOT.RooDataHist("data", "dataset with mass", ROOT.RooArgList(mass), hist)


In [ ]:
## Sinal PDF : Crystal Ball + Gaussian
frac_gauss = ROOT.RooRealVar("frac_gauss", "", 0.4, 0.0, 1.0)
mean = ROOT.RooRealVar("mean", "",jpsi_pdg_mass , m_min, m_max)
sigma_gauss = ROOT.RooRealVar("sigma gauss", "",0.019 , 0.000001, 1.0)

frac_cb = ROOT.RooRealVar("frac_cb", "", 0.6, 0.0, 1.0)
sigma_cb = ROOT.RooRealVar("sigma cb", "",0.019 , 0.000001, 1.0)
alpha = ROOT.RooRealVar("alpha", "",1.4 , 0.0, 10.0)
n = ROOT.RooRealVar("n", "", 20 , 0, 150)
n.setConstant(True)

signal1 = ROOT.RooGaussian("gauss", "", mass, mean, sigma_gauss)
signal2 = ROOT.RooCBShape("crystal ball", "", mass, mean, sigma_cb,alpha,n)


In [ ]:
#Background PDF Model
exp_c0 = RooRealVar("exp_c0", "", -3,-5,5)
bkg = ROOT.RooExponential("bkg", "",mass,exp_c0)



In [ ]:
model = ROOT.RooAddPdf("model", "", ROOT.RooArgList(signal1,signal2,bkg),ROOT.RooArgList(frac_gauss,frac_cb), ROOT.kTRUE)

In [ ]:
fit_result = model.fitTo(data, ROOT.RooFit.Save())

In [ ]:
colors = {"model" : 2, "signal" : 4, "background" : 3}
styles = {"model" : 1, "signal" : 1, "background" : 2}

ca = ROOT.TCanvas("can", "",900,600)

frame = mass.frame(ROOT.RooFit.Title("J/\Psi Mass Fit"), ROOT.RooFit.Bins(100))
frame.GetXaxis().SetTitle(r'm_{\mu^+\mu^-} [GeV/c^2]')

# Data
data.plotOn(frame, ROOT.RooFit.Name("Data"), ROOT.RooFit.DataError(ROOT.RooAbsData.SumW2))

# Signal
model.plotOn(frame, ROOT.RooFit.Name("Signal"), ROOT.RooFit.Components("gauss,crystal ball"), ROOT.RooFit.LineStyle(styles["signal"]), ROOT.RooFit.LineColor(colors["signal"]))

# Background
model.plotOn(frame, ROOT.RooFit.Name("Background"), ROOT.RooFit.Components("bkg"), ROOT.RooFit.LineStyle(styles["background"]), ROOT.RooFit.LineColor(colors["background"]))

# Model
model.plotOn(frame, ROOT.RooFit.Name("Model"), ROOT.RooFit.LineStyle(styles["model"]), ROOT.RooFit.LineColor(colors["model"]))

frame.Print()

# Extraindo os parâmetros ajustados e seus erros
mean_val = mean.getValV()
mean_err = mean.getError()
sigma_val = sigma_gauss.getValV()
sigma_err = sigma_gauss.getError()

import math

# Cálculo do número total de eventos (n_total)
n_total = data.sumEntries()

# Cálculo do yields do sinal
frac_cb_val = frac_cb.getVal()  # Valor de frac_cb
frac_gauss_val = frac_gauss.getVal()  # Valor de frac_gauss
frac_cb_err = frac_cb.getError()
frac_gauss_err = frac_gauss.getError()

# Número de eventos de sinal
nsignal = n_total * (frac_cb_val + (1 - frac_cb_val) * frac_gauss_val)
# Propagação de erros para o sinal
nsignal_err = n_total * math.sqrt((frac_cb_err)**2 + ((1 - frac_cb_val) * frac_gauss_err)**2)

# Número de eventos de fundo (N_total - N_signal)
#nbackground = n_total - nsignal ou N_back = n_total (1 - fgauss) * (1 - fcrystall) * fexponential

nbackground = n_total * (1 - frac_gauss_val) * (1 - frac_cb_val)

# Propagação de erros para o fundo
nbackground_err = n_total * math.sqrt(
    ((1 - frac_cb_val) * frac_gauss_err) ** 2 +
    ((1 - frac_gauss_val) * frac_cb_err) ** 2
)


## ChiSquare computation
n_param = fit_result.floatParsFinal().getSize()
print("nr de par:", n_param)
reduce_chi_square = frame.chiSquare(n_param)
print("chi sqare:", frame.chiSquare() )
print("chi_square/n_param", reduce_chi_square )

frame.Draw()

## Legendas

#legend_text = ROOT.TPaveText(0.65, 0.6, 0.9, 0.85, "NDC")
#legend_text = ROOT.TPaveText(0.75, 0.25, 0.9, 0.35, "NDC")
legend_text = ROOT.TPaveText(0.7, 0.55, 0.88, 0.7, "NDC")
legend_text.AddText(f"Mean = {mean_val:.4f} #pm {mean_err:.4f} GeV/c^2")
legend_text.AddText(f"Sigma = {sigma_val:.4f} #pm {sigma_err:.4f} GeV/c^2")
# Adicionar o rendimento do sinal e do fundo
legend_text.AddText(f"Signal Yield = {nsignal:.0f} #pm {nsignal_err:.0f}")
legend_text.AddText(f"Background Yield = {nbackground:.0f} #pm {nbackground_err:.0f}")
#legend_text.AddText(f"nr de param = {n_param:.2f}")
legend_text.AddText(f"Chi2/ndof = {reduce_chi_square:.2f}")

legend_text.SetFillColor(0)
legend_text.SetBorderSize(1)
legend_text.Draw()

#leg = ROOT.TLegend(0.7, 0.7, 0.88, 0.89)
leg = ROOT.TLegend(0.7, 0.75, 0.88, 0.89)
leg.AddEntry(frame.findObject("Data"), "Data", "LEP")
leg.AddEntry(frame.findObject("Model"), "Model Fit", "L")
leg.AddEntry(frame.findObject("Signal"), "Signal Fit", "L")
leg.AddEntry(frame.findObject("Background"), "Background fit", "L")



leg.Draw("same")
ca.Update()
ca.Draw()

ca.SaveAs("M_JPsi_fit.png")

In [ ]:
fit_result.Print()

O pull é uma medida de quão bem o seu modelo ajustado se alinha com os dados. Para cada bin, o pull representa a diferença entre o valor observado (dados) e o valor previsto pelo ajuste (modelo), normalizado pelo erro. Idealmente, a distribuição de pull deve ser centrada em torno de zero com um desvio padrão próximo de 1.

In [ ]:
''''
# Crie um frame a partir da variável 'mass' que está sendo ajustada
mass_frame = mass.frame(ROOT.RooFit.Title("Fit Results"))

# Plote os dados no frame
data.plotOn(mass_frame)

# Plote o modelo ajustado no frame
model.plotOn(mass_frame)

# Agora crie o histograma de pulls a partir do gráfico de ajuste
histpull = mass_frame.pullHist()

# Crie um novo frame para a distribuição de pulls
pull_frame = mass.frame(ROOT.RooFit.Title("Pull Distribution"))
pull_frame.addPlotable(histpull, "P")


can2 = ROOT.TCanvas("can2", "Pull Distribution", 900, 300)

# Ajuste o frame para o pull
pull_frame.GetXaxis().SetTitle("m_{#mu^+\mu^-} [GeV/c^2]")
pull_frame.GetYaxis().SetTitle("Pull")
pull_frame.GetYaxis().SetTitleOffset(1.6)

# Desenhe o gráfico de pull
can2.cd()
pull_frame.Draw()

# Calcular o mean e o desvio padrão dos pulls
mean_hist = histpull.GetMean()
stddev_hist = histpull.GetRMS()

mean_pull = np.mean(histpull.GetY())  # Calcula o valor médio (mean) dos pulls
sigma_pull = np.std(histpull.GetY())  # Calcula o desvio padrão (sigma) dos pulls
print(f"Mean of Pull: {mean_pull}")
print(f"Sigma (Standard Deviation) of Pull: {sigma_pull}")
# Adicionando o texto com o mean e o desvio padrão no gráfico de pull
latex = ROOT.TLatex()
latex.SetNDC()  # Para coordenadas normalizadas

# Desenhe o mean e stddev no gráfico
latex.DrawLatex(0.15, 0.85, f"Mean: {mean_pull:.2f}")
latex.DrawLatex(0.15, 0.80, f"Std Dev: {sigma_pull:.2f}")

# Exibir o canvas com a informação correta
can2.Draw()
'''


In [ ]:
# Workspace
wspace = ROOT.RooWorkspace("Jpsi fit")

getattr(wspace, "import")(data)
getattr(wspace, "import")(model)

wspace.writeToFile("Jpsi_fit.root")

#ROOT.DisableImplicitMT()